In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import glob

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
results_file = 'write/tsukui_ali.h5ad'  # the file that will store the analysis results

In [ ]:
filenames = (glob.glob('/mnt/cip/home/sch/schuboea/tsukuidata/*.mtx'))
samplenames = []
for file in filenames:
    sample = '_'.join((file.split(sep="/")[-1]).split(sep="_")[0:-1])
    samplenames.append(sample)
print(samplenames)

In [ ]:
#generate adata object for each sample matrix and store them in "adatas"-list
adatas = []
for sample in samplenames:
    adata = sc.read_10x_mtx(
    '/mnt/cip/home/sch/schuboea/tsukuidata/',  # the directory with the `.mtx` file
    prefix = sample + "_",                # sample name as prefix
    var_names='gene_symbols',             # use gene symbols for the variable names (variables-axis index)
    cache=True)                          # write a cache file for faster subsequent reading
    adata.var_names_make_unique()        # make sure genes with same name don´t cause trouble
    adatas.append(adata)
print(adatas)

In [ ]:
#hier direkt concat, davor noch obs columns für condition, batch, und gfp y/n reinmachen
#dann qc machen auf allen
#plots in seaborn direkt nebeneinander mit der gleichen achse (wie beim best practice buch -> histogramme), auch mal schauen wegen <2500
#<3500 und mt <6

In [ ]:
#adata = adatas[1]
counter=0
for sample in samplenames:
    #adata = "adatas[counter]"
    
    #sc.pp.filter_cells(adatas[counter], min_genes=250)  #filter out cells with less than 250 genes (from publication), usually it´s 200?
    sc.pp.filter_genes(adatas[counter], min_cells=3) #filter out genes that are seen in less than 3 cells
    
    # mitochondrial genes
    adatas[counter].var["mt"] = adatas[counter].var_names.str.startswith("mt-")
    sc.pp.calculate_qc_metrics(adatas[counter], qc_vars=["mt"], inplace=True, percent_top=None, log1p=True) 
    sc.pl.violin(adatas[counter], [ 'total_counts', 'pct_counts_mt', 'n_genes_by_counts'],
                 jitter=0.4, multi_panel=True)
    sc.pl.scatter(adatas[counter], x='total_counts', y='n_genes_by_counts')
    sc.pl.scatter(adatas[counter], x='total_counts', y='pct_counts_mt')
    
    adatas[counter] = adatas[counter][adatas[counter].obs.n_genes_by_counts < 2500, :]
    adatas[counter] = adatas[counter][adatas[counter].obs.pct_counts_mt < 10, :] #Hier percent gegen [5] für standard oder [10] von pub tauschen (oder MAD nehmen wie in best practices)
    
    #add obs with condition, batch and gfp y/n info
    adata_copy = adatas[counter].copy()
    adata_copy.obs["batch"]=sample
    adata_copy.obs["condition"]=(sample.split(sep="_")[1])[:-1]
    adata_copy.obs["gfp"]=sample.split(sep="_")[2]
    adatas[counter] = adata_copy
    #print(adata.var)
    #print(adata.obs)
    counter+=1

In [ ]:
big_adata = adatas[0].concatenate(adatas[1:], join="outer") #index_unique=None?

In [ ]:
print(big_adata)

In [ ]:
adatas[1].obs

In [ ]:
#preprocessing for each adata object in the list
for adata in adatas:
    sc.pp.filter_cells(adata, min_genes=250)  #filter out genes with less than 250 genes (from publication)
    '''sc.pp.filter_cells(ts, min_genes=300)
    sc.pp.filter_cells(ts, max_genes=4600)     #daniels cut offs
    sc.pp.filter_cells(ts, max_counts=20000)'''
    # mitochondrial genes
    adata.var["mt"] = adata.var_names.str.startswith("mt-")
    # ribosomal genes
    adata.var["ribo"] = adata.var_names.str.startswith(("rps", "rpl"))
    # hemoglobin genes.
    adata.var["hb"] = adata.var_names.str.contains(("^hb[^(p)]"))
    sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[10], log1p=True)
    adata